Group Homework Members: Çağatay Karabulut, Ayberk Akgün, Gülce Karabacak

In [110]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In this task, we decided to use 9 features to predict and classify matches. For a single match we use average of goals scored and goals conceded of the last 5 home games and last 5 away games for both teams. The other feature is the average of the total goals for last 3 encounters of these teams. These features will be the indicator for both teams' offence and defence performance and the last feature will be the indicator for this encounter in particular. 

In [3]:
def avg_home_score(team_id, n_matches, for_epoch):
    matches_grouped_home = matches[['match_hometeam_id','match_id','epoch','match_hometeam_score']].loc[(matches.epoch < for_epoch) & (matches.match_hometeam_id == team_id)]
    return matches_grouped_home.nlargest(n_matches,'epoch').agg({'match_hometeam_score':'mean'})[0]

def avg_away_score(team_id, n_matches, for_epoch):
    matches_grouped_home = matches[['match_awayteam_id','match_id','epoch','match_awayteam_score']].loc[(matches.epoch < for_epoch) & (matches.match_awayteam_id == team_id)]
    return matches_grouped_home.nlargest(n_matches,'epoch').agg({'match_awayteam_score':'mean'})[0]

def avg_home_concede(team_id, n_matches, for_epoch):
    matches_grouped_home = matches[['match_hometeam_id','match_id','epoch','match_awayteam_score']].loc[(matches.epoch < for_epoch) & (matches.match_hometeam_id == team_id)]
    return matches_grouped_home.nlargest(n_matches,'epoch').agg({'match_awayteam_score':'mean'})[0]

def avg_away_concede(team_id, n_matches, for_epoch):
    matches_grouped_home = matches[['match_awayteam_id','match_id','epoch','match_hometeam_score']].loc[(matches.epoch < for_epoch) & (matches.match_awayteam_id == team_id)]
    return matches_grouped_home.nlargest(n_matches,'epoch').agg({'match_hometeam_score':'mean'})[0]

def last_game_score(home_id, away_id, n_matches, for_epoch):
    matches_grouped = matches[['match_hometeam_id','match_awayteam_id','epoch','match_hometeam_score','match_awayteam_score']].loc[matches.epoch < for_epoch].loc[(((matches.match_awayteam_id == away_id) & (matches.match_hometeam_id == home_id)) | ((matches.match_hometeam_id == away_id) & (matches.match_awayteam_id == home_id)))]
    return matches_grouped.nlargest(n_matches,'epoch').agg({'match_hometeam_score':'mean','match_awayteam_score':'mean'})

Nowi I am reading csv files.

In [ ]:
bets = pd.read_csv(r"C:\Users\cagatay\Desktop\IE_582_Final_Project\bets.csv")
booking = pd.read_csv(r"C:\Users\cagatay\Desktop\IE_582_Final_Project\booking.csv")
goals = pd.read_csv(r"C:\Users\cagatay\Desktop\IE_582_Final_Project\goals.csv")
matches= pd.read_csv(r"C:\Users\cagatay\Desktop\IE_582_Final_Project\matches.csv")
stats = pd.read_csv(r"C:\Users\cagatay\Desktop\IE_582_Final_Project\stats.csv")

In [5]:
bets.head(3)

,match_id,odd_bookmakers,odd_epoch,variable,value
0,146845,BetOlimp,1486301854,odd_1,1.96
1,151780,10Bet,1486314920,odd_1,2.15
2,151780,18bet,1486314920,odd_1,2.17


In [6]:
booking.head(3)

,match_id,time,home_fault,card,away_fault
0,13327,90+3,Can E.,yellow card,NaN
1,13329,29,Holebas J.,yellow card,NaN
2,13329,40,Doucoure A.,yellow card,NaN


In [7]:
goals.head(3)

,match_id,time,home_scorer,score,away_scorer
0,13327,30,Salah M.,1 - 1,NaN
1,13446,35,Colin M.,1 - 0,NaN
2,13446,56,NaN,1 - 1,Johnson D.


In [8]:
matches.head(3)

,match_awayteam_id,match_hometeam_id,match_id,epoch,match_status,match_live,match_hometeam_name,match_awayteam_name,match_hometeam_score,match_awayteam_score,match_hometeam_halftime_score,match_awayteam_halftime_score,match_hometeam_extra_score,match_awayteam_extra_score,match_hometeam_penalty_score,match_awayteam_penalty_score,league_id
0,7109,7097,41196,1505559600,Finished,0,Levante,Valencia,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,468
1,2614,2619,13331,1505561400,Finished,0,Crystal Palace,Southampton,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,148
2,3224,3238,17683,1505568600,Finished,0,Eintracht Frankfurt,FC Augsburg,1.0,2.0,0.0,1.0,0.0,1.0,NaN,NaN,195


In [9]:
stats.head(3)

,match_id,home_BallPossession,home_CornerKicks,home_Fouls,home_GoalAttempts,home_GoalkeeperSaves,home_Offsides,home_ShotsoffGoal,home_ShotsonGoal,home_YellowCards,...,home_DistanceCovered(metres),away_DistanceCovered(metres),home_PassSuccess%,away_PassSuccess%,home_Attacks,home_DangerousAttacks,away_Attacks,away_DangerousAttacks,home_,away_
0,13327,71%,12.0,7.0,35.0,3.0,1.0,14.0,9.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13329,33%,3.0,8.0,7.0,4.0,2.0,5.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13331,45%,5.0,14.0,14.0,3.0,0.0,9.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We formed feature list and then turned it to dataframe. 

In [10]:
feature_list = []
for index, i in matches.iterrows():
    feature_list.append([i.match_id, 
                         i.match_hometeam_id, 
                         avg_home_score(i.match_hometeam_id,5,i.epoch), avg_away_score(i.match_hometeam_id,5,i.epoch), 
                         avg_home_concede(i.match_hometeam_id,5,i.epoch), avg_away_concede(i.match_hometeam_id,5,i.epoch),
                         i.match_awayteam_id, 
                         avg_home_score(i.match_awayteam_id,5,i.epoch), avg_away_score(i.match_awayteam_id,5,i.epoch),
                         avg_home_concede(i.match_awayteam_id,5,i.epoch), avg_away_concede(i.match_awayteam_id,5,i.epoch),
                         last_game_score(i.match_hometeam_id, i.match_awayteam_id, 3, i.epoch).sum(),
                         i.match_hometeam_score + i.match_awayteam_score])

In [11]:
feature_df = pd.DataFrame(feature_list, columns = ['match_id',
                                                   'home_id',
                                                   'hometeam_avg_home_score','hometeam_avg_away_score',
                                                   'hometeam_avg_home_concede','hometeam_avg_away_concede',
                                                   'away_id',
                                                   'awayteam_avg_home_score','awayteam_avg_away_score',
                                                   'awayteam_avg_home_concede','awayteam_avg_away_concede',
                                                   'avg_score_between',
                                                   'total_goals'])
feature_df.dropna(inplace = True)
feature_df.reset_index(inplace = True, drop = True)

In [12]:
feature_df.tail()

,match_id,home_id,hometeam_avg_home_score,hometeam_avg_away_score,hometeam_avg_home_concede,hometeam_avg_away_concede,away_id,awayteam_avg_home_score,awayteam_avg_away_score,awayteam_avg_home_concede,awayteam_avg_away_concede,avg_score_between,total_goals
5310,242326,2645,1.8,1.2,2.2,2.0,2634,1.0,1.2,0.6,1.0,3.000000,4.0
5311,242324,2633,1.4,2.0,1.2,1.2,2643,3.0,0.6,1.6,0.6,2.666667,4.0
5312,242323,2680,1.8,1.4,1.6,2.2,2644,1.0,2.4,0.8,0.6,4.000000,1.0
5313,242325,2648,0.8,0.6,1.6,1.4,2652,1.6,1.2,2.4,2.6,4.000000,1.0
5314,242328,2625,2.8,1.2,1.8,0.2,2650,1.0,1.4,0.6,1.8,5.500000,5.0


Since we will use xgboost library for stochastic gradient boosting, we turned the features and labels to dmatrix.

In [94]:
feature_columns = ['hometeam_avg_home_score','hometeam_avg_away_score',
                   'hometeam_avg_home_concede','hometeam_avg_away_concede',
                   'awayteam_avg_home_score','awayteam_avg_away_score',
                   'awayteam_avg_home_concede','awayteam_avg_away_concede',
                   'avg_score_between']

X_train, X_test, y_train, y_test = train_test_split(feature_df[feature_columns], feature_df['total_goals'])

feature_dmatrix = xgboost.DMatrix(data = np.array(X_train), label = np.array(y_train), feature_names = feature_columns)
test_dmatrix = xgboost.DMatrix(data = np.array(X_test), label = np.array(y_test), feature_names = feature_columns)

Then, we are creating regressor and classifier of xgboost library.

In [14]:
sgb_regressor = xgboost.XGBRegressor(min_child_weight=10)
sgb_classifier = xgboost.XGBClassifier(min_child_weight=10)

We created combinations of parameters to find the best parameter set for each model.

In [59]:
param_grid_sgb = dict(max_depth = [2,3,5,7,10],
                  learning_rate = [0.1, 0.2, 0.5, 0.7],
                  n_estimators = [10,30,50,100,200])
param_grid_tree = dict(ccp_alpha = [0.001, 0.002, 0.003, 0.004], 
                       min_samples_leaf = [2,3,5,7,10])
param_grid_rf = dict(n_estimators = [500],
                     min_samples_leaf = [5],
                     max_features = [2,3,4,5])

Kfold object is created.

In [16]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)

Grid search model is composed with a kfold object, a parameter grid object and a regressor or a classifier. 

In [17]:
grid_search = GridSearchCV(sgb_regressor, param_grid_sgb, cv = kfold )
grid_search_clf = GridSearchCV(sgb_classifier, param_grid_sgb, cv = kfold)

Models are trained.

In [ ]:
results_sgb_reg = grid_search.fit(np.array(X_train), np.array(y_train))
results_sgb_clf = grid_search_clf.fit(np.array(X_train), np.array((y_train > 2.5).astype('int')))

LassoCV model of sklearn library tries alpha values on its own and gives the best estimate. 

In [81]:
lasso_reg = LassoCV(cv = kfold)
lasso_reg = lasso_reg.fit(np.array(X_train), np.array(y_train))

Classifiers and regressors of both tree and randomforest are defined.

In [24]:
tree_clf = DecisionTreeClassifier()
tree_reg = DecisionTreeRegressor()

rf_clf = RandomForestClassifier()
rf_reg = RandomForestRegressor()

Grid search object are created.

In [60]:
grid_search_tree_clf = GridSearchCV(tree_clf, param_grid_tree, cv = kfold )
grid_search_tree_reg = GridSearchCV(tree_reg, param_grid_tree, cv = kfold)

grid_search_rf_clf = GridSearchCV(rf_clf, param_grid_rf, cv = kfold )
grid_search_rf_reg = GridSearchCV(rf_reg, param_grid_rf, cv = kfold)

Models are trained and best parameters are found. 

In [ ]:
results_tree_reg = grid_search_tree_reg.fit(np.array(X_train), np.array(y_train))
results_tree_clf = grid_search_tree_clf.fit(np.array(X_train), np.array(y_train > 2.5).astype('int')))

results_rf_reg = grid_search_rf_reg.fit(np.array(X_train), np.array(y_train))
results_rf_clf = grid_search_rf_clf.fit(np.array(X_train), np.array((y_train > 2.5).astype('int')))

Below shows the best parameters for 4 models

In [82]:
results_tree_clf.best_params_

{'ccp_alpha': 0.001, 'min_samples_leaf': 2}

In [106]:
results_rf_clf.best_params_

{'max_features': 4, 'min_samples_leaf': 5, 'n_estimators': 500}

In [84]:
results_sgb_clf.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}

In [85]:
results_tree_reg.best_params_

{'ccp_alpha': 0.004, 'min_samples_leaf': 10}

In [86]:
results_rf_reg.best_params_

{'max_features': 2, 'min_samples_leaf': 5, 'n_estimators': 500}

In [87]:
results_sgb_reg.best_params_

{'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 30}

In [89]:
lasso_reg.alpha_

0.00017487286368655922

In [103]:
print(accuracy_score(results_sgb_clf.best_estimator_.predict(np.array(X_test)), (y_test > 2.5).astype('int')))
print(results_sgb_clf.best_score_)

0.5914221218961625
0.5546983035478142


In [104]:
print(accuracy_score(results_rf_clf.best_estimator_.predict(np.array(X_test)), (y_test > 2.5).astype('int')))
print(results_rf_clf.best_score_)

0.9661399548532731
0.5554040468402645


In [105]:
print(accuracy_score(results_tree_clf.best_estimator_.predict(np.array(X_test)), (y_test > 2.5).astype('int')))
print(results_tree_clf.best_score_)

0.5658389766741911
0.5537179813941634


In [114]:
print(r2_score(results_sgb_reg.best_estimator_.predict(np.array(X_test)), y_test) / len(y_test))
print(results_sgb_reg.best_score_)

-0.02224615724078099
0.02407666014643871


In [115]:
print(r2_score(results_rf_reg.best_estimator_.predict(np.array(X_test)), y_test) / len(y_test))
print(results_rf_reg.best_score_)

-0.0047822305363527185
0.01445504580432574


In [116]:
print(r2_score(results_tree_reg.best_estimator_.predict(np.array(X_test)), y_test) / len(y_test))
print(results_tree_reg.best_score_)

-0.014070245859383754
-0.015498281547532022


In [118]:
print(r2_score(lasso_reg.predict(np.array(X_test)), y_test) / len(y_test))
print(results_tree_reg.best_score_)

-0.021904986625712476
-0.015498281547532022


Except for random forest classifier, others have similar cross validation and test score values. However it doesn't mean that classifiers are good at estimation in regression, we can say it by looking their r2 scores and accuracy scores.
Similar error values and bad accuracy results shows that features are not good at prediction labels. For random forest case, cross validation and test accuracy differs because of overfitting. 

Lasso regression has the smallest alpha possible because as there are very few features with low predictive capabilities, the model can't discard any of them